In [1]:
# run python scripts containing all the functions
%run -i '0_paths_open_save.py'
%run -i '1_scraping_cleaning_merge.py'
%run -i '2_summary_stats_visualization_new.py'

# 1. check scraped data

In [ ]:
initial_panel = '201812'
tracking_panels = ['201812', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102']

o = open_files(initial_panel=initial_panel, all_panels=tracking_panels)
app_details = o.open_initial_panel_with_its_tracking_panels()
A = app_detail_dicts(app_details, all_panels=tracking_panels)
A.get_a_glimpse()

# 2. check converted data

In [ ]:
# test
%run -i '1_scraping_cleaning_merge.py'
initial_panel='201907'
all_panels=['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102']
o = open_files(initial_panel=initial_panel,
                   all_panels=all_panels)
app_details = o.open_initial_panel_with_its_tracking_panels()
A = app_detail_dicts(d=app_details, all_panels=all_panels)
inter_df = A.format_cols()
print(inter_df['201907'].columns)


# 3. check whether the developer geocoding information is correct.

In [76]:
%run -i '0_paths_open_save.py'
%run -i '1_scraping_cleaning_merge.py'
%run -i '2_summary_stats_visualization_new.py'
initial_panel='201907'
o = open_files(initial_panel=initial_panel)
DF = o.open_panel_df(type='dev_index_geocoded')
all_panels = ['201907',
              '201908',
              '201909',
              '201912',
              '202001',
              '202003',
              '202004',
              '202009',
              '202010',
              '202011',
              '202012',
              '202101',
              '202102']
consec_panels = ['202009', '202010', '202011', '202012', '202101', '202102']
new_class_obj = pre_processing(
                  df=DF,
                  df_index = 'dev_index_geocoded',
                  initial_panel=initial_panel,
                  all_panels=all_panels,
                  consec_panels=consec_panels)
print(new_class_obj.df.shape)
DF2 = new_class_obj.df[['developerAddress_201907', 'location', 'longitude', 'latitude']]
testdf1 = DF2.sample(100)
testdf1

(6442, 55)


,developerAddress_201907,location,longitude,latitude
developer,,,,
My Apps Studio,None,None,NaN,NaN
Pintogykas,None,None,NaN,NaN
International Dairy Queen®️,"8331 Norman Center Drive\nBloomington, MN 55437","(8331 Norman Center Drive, Minneapolis, MN 55437, (44.85294, -93.35164))",-93.35164,44.85294
"SNOW, Inc.","575 High Street,Suite 110 Palo Alto, CA 94301","(575 High Street, Palo Alto, CA 94301, (37.44357, -122.16224))",-122.16224,37.44357
Saucon IoT,None,None,NaN,NaN
...,...,...,...,...
Straight Talk,None,None,NaN,NaN
Maadinfo Services,2 allée des Ormeaux\n06130 Grasse\nFrance,"(2 Allée des Ormeaux, Grasse, 06130, (43.6564, 6.91355))",6.91355,43.65640
Usatine Media,"8529 Raintree Woods Drive\nFair Oaks Ranch, TX 78255","(8529 Raintree Woods Drive, Boerne, TX 78015, (29.73904, -98.65005))",-98.65005,29.73904


# 4. check whether the multiindex conversion is correct

In [ ]:
# test file
%run -i '0_paths_open_save.py'
%run -i '1_scraping_cleaning_merge.py'
%run -i '2_summary_stats_visualization_new.py'

o = open_files(initial_panel='201812')
DF = o.open_panel_df(type='dev_multiindex_geocoded')
all_panels = ['201812',
              '201912',
              '202001',
              '202003',
              '202004',
              '202009',
              '202010',
              '202011',
              '202012',
              '202101',
              '202102']
consec_panels = ['202009', '202010', '202011', '202012', '202101', '202102']
new_class_obj = pre_processing(
                  df=testdf1,
                  df_index = 'dev_multiindex_geocoded',
                  initial_panel='201812',
                  all_panels=all_panels,
                  consec_panels=consec_panels)
new_class_obj.print_col_names(text='appId')

# 5. test code for deleting missing, imputing missing and deleting outliers

In [177]:
# test file
%run -i '0_paths_open_save.py'
%run -i '1_scraping_cleaning_merge.py'
%run -i '2_summary_stats_visualization_new.py'
initial_panel='201907'
o = open_files(initial_panel=initial_panel)
DF = o.open_panel_df(type='MERGED')
all_panels = ['201907',
              '201908',
              '201909',
              '201912',
              '202001',
              '202003',
              '202004',
              '202009',
              '202010',
              '202011',
              '202012',
              '202101',
              '202102']
consec_panels = ['202009', '202010', '202011', '202012', '202101', '202102']
new_class_obj = pre_processing(
                  df=DF,
                  df_index = 'appid',
                  initial_panel=initial_panel,
                  all_panels=all_panels,
                  consec_panels=consec_panels)
print(new_class_obj.df.shape)

(10576, 396)


In [5]:
new_class_obj.print_col_names()

title_201907
description_201907
summary_201907
minInstalls_201907
score_201907
ratings_201907
reviews_201907
histogram_201907
price_201907
currency_201907
size_201907
developer_201907
released_201907
updated_201907
comments_201907
developerId_201907
developerWebsite_201907
developerEmail_201907
developerAddress_201907
appId_201907
adSupported_True_201907
containsAds_True_201907
free_True_201907
offersIAP_True_201907
contentRating_Adult_201907
contentRating_Everyone_201907
contentRating_Teen_201907
genreId_Entertainment_201907
genreId_Game_201907
genreId_Productivity_201907
title_201908
description_201908
summary_201908
minInstalls_201908
score_201908
ratings_201908
reviews_201908
histogram_201908
price_201908
currency_201908
size_201908
developer_201908
released_201908
updated_201908
comments_201908
developerId_201908
developerWebsite_201908
developerEmail_201908
developerAddress_201908
appId_201908
adSupported_True_201908
containsAds_True_201908
free_True_201908
offersIAP_True_201908


Index(['title_201907', 'description_201907', 'summary_201907',
       'minInstalls_201907', 'score_201907', 'ratings_201907',
       'reviews_201907', 'histogram_201907', 'price_201907', 'currency_201907',
       ...
       'adSupported_True_202102', 'containsAds_True_202102',
       'free_True_202102', 'offersIAP_True_202102',
       'contentRating_Adult_202102', 'contentRating_Everyone_202102',
       'contentRating_Teen_202102', 'genreId_Entertainment_202102',
       'genreId_Game_202102', 'genreId_Productivity_202102'],
      dtype='object', length=396)

In [174]:
missingdf = new_class_obj.count_missing(var_list=['minInstalls', 'score', 'reviews',
                                                  'ratings', 'price', 'description',
                                                  'released', 'updated', 'size', 'free_True', 'adSupported_True',
                                                  'containsAds_True', 'offersIAP_True', 'contentRating_Adult',
                                                  'contentRating_Everyone', 'contentRating_Teen',
                                                  'genreId_Entertainment', 'genreId_Game',
                                                  'genreId_Productivity'])
print(missingdf)

                               count missing
minInstalls_201907             25           
minInstalls_201908             25           
minInstalls_201909             25           
minInstalls_201912             25           
minInstalls_202001             25           
minInstalls_202003             25           
minInstalls_202004             25           
minInstalls_202009             50           
minInstalls_202010             113          
minInstalls_202011             167          
minInstalls_202012             287          
minInstalls_202101             336          
minInstalls_202102             396          
score_201907                   8            
score_201908                   8            
score_201909                   8            
score_201912                   8            
score_202001                   8            
score_202003                   8            
score_202004                   8            
score_202009                   54           
score_2020

In [179]:
missing_df, missing_appids = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='size',
    number_consec_panels_missing=6)
print(missing_df.shape)

number of apps with at least 6 consecutive missing panels for size are 1849
out of 10576 apps.
(13, 1849)


In [175]:
# reproduce the method impute_missing_using_adj_panels
cols = []
for i in new_class_obj.df.columns:
    if 'size_' in i:
        cols.append(i)
test_df = new_class_obj.df[cols]
test_df = test_df.loc[['com.redberry.glitterlips',
         'zonek.design.paintbynumbers',
         'com.piupiuapps.hairstylesgirls']]
test_df

,size_201907,size_201908,size_201909,size_201912,size_202001,size_202003,size_202004,size_202009,size_202010,size_202011,size_202012,size_202101,size_202102
com.redberry.glitterlips,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,34.0,NaN,34.0
zonek.design.paintbynumbers,58.0,58.0,58.0,58.0,58.0,58.0,58.0,58.0,58.0,58.0,NaN,NaN,NaN
com.piupiuapps.hairstylesgirls,16.0,16.0,16.0,16.0,16.0,16.0,16.0,15.0,NaN,NaN,NaN,NaN,NaN


In [176]:
import functools
import copy
adj_panels = 7
method = 'mean'
df_list = []
for j in range(len(test_df.columns)):
    if j <= adj_panels//2 or j in [0, 1]:
        df = test_df.iloc[:, 0:adj_panels+1]
    elif j >= len(test_df.columns)-adj_panels//2-1:
        df = test_df.iloc[:, len(test_df.columns)-adj_panels-1:len(test_df.columns)]
    else:
        if adj_panels == 1:
            df = test_df.iloc[:, j-1:j+1]
        else:
            df = test_df.iloc[:, j-adj_panels//2:j+adj_panels//2+1]
    if method == 'mean':
        df[method] = df.mean(axis=1, skipna=True)
    elif method == 'mode':
        df[method] = df.mode(axis=1, numeric_only=False, dropna=True).iloc[:,0]
    elif method == 'previous':
        df[method] = df.iloc[:,0]
    else:
        df[method] = 0
    dfd = copy.deepcopy(df)
    for col in dfd.columns:
        dfd.loc[dfd[col].isnull(), col] = dfd[method]
    dfd = dfd[[test_df.columns[j]]]
    df_list.append(dfd)
imputed_df = functools.reduce(lambda a, b: a.join(b, how='inner'), df_list)
imputed_df

,size_201907,size_201908,size_201909,size_201912,size_202001,size_202003,size_202004,size_202009,size_202010,size_202011,size_202012,size_202101,size_202102
com.redberry.glitterlips,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.000000,33.000000,34.000000,33.285714,34.000000
zonek.design.paintbynumbers,58.0,58.0,58.0,58.0,58.0,58.0,58.0,58.0,58.000000,58.000000,58.000000,58.000000,58.000000
com.piupiuapps.hairstylesgirls,16.0,16.0,16.0,16.0,16.0,16.0,16.0,15.0,15.666667,15.666667,15.666667,15.666667,15.666667


In [31]:
# impute updated
missing_df_updated, missing_appids_updated = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='updated',
    number_consec_panels_missing=1)
print(missing_df_updated.shape)

DFT = new_class_obj.impute_missing_using_adj_panels(
    var='updated',
    adj_panels=1,
    method='mode')
new_DF = new_class_obj.replace_cols(new_cols=DFT)

number of apps with at least 1 consecutive missing panels for updated are 528
out of 10576 apps.
(13, 528)


In [34]:
# check imputed updated
missing_df_updated[['com.redberry.glitterlips',
         'zonek.design.paintbynumbers',
         'com.piupiuapps.hairstylesgirls']]

,com.redberry.glitterlips,zonek.design.paintbynumbers,com.piupiuapps.hairstylesgirls
updated_201907,2019-12-30,2019-04-04,2020-01-24
updated_201908,2019-12-30,2019-04-04,2020-01-24
updated_201909,2019-12-30,2019-04-04,2020-01-24
updated_201912,2019-12-30,2019-04-04,2020-01-24
updated_202001,2019-12-30,2019-04-04,2020-01-24
updated_202003,2019-12-30,2019-04-04,2020-01-24
updated_202004,2019-12-30,2019-04-04,2020-01-24
updated_202009,2019-12-30,2019-04-04,2020-07-15
updated_202010,2020-09-15,2019-04-04,NaT
updated_202011,2020-09-15,2019-04-04,NaT


,updated_201907,updated_201908,updated_201909,updated_201912,updated_202001,updated_202003,updated_202004,updated_202009,updated_202010,updated_202011,updated_202012,updated_202101,updated_202102
com.redberry.glitterlips,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2020-09-15,2020-09-15,2020-09-15,NaT,2020-09-15
zonek.design.paintbynumbers,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,NaT,NaT,NaT
com.piupiuapps.hairstylesgirls,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-07-15,NaT,NaT,NaT,NaT,NaT


In [170]:
df_list[12]

,updated_202101,updated_202102,previous
com.redberry.glitterlips,NaT,2020-09-15,NaT
zonek.design.paintbynumbers,NaT,NaT,NaT
com.piupiuapps.hairstylesgirls,NaT,NaT,NaT


In [35]:
# check imputed updated
DFT.loc[['com.redberry.glitterlips',
         'zonek.design.paintbynumbers',
         'com.piupiuapps.hairstylesgirls']]

,updated_201907,updated_201908,updated_201909,updated_201912,updated_202001,updated_202003,updated_202004,updated_202009,updated_202010,updated_202011,updated_202012,updated_202101,updated_202102
com.redberry.glitterlips,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2019-12-30,2020-09-15,2020-09-15,2020-09-15,2020-09-15,2020-09-15
zonek.design.paintbynumbers,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04,2019-04-04
com.piupiuapps.hairstylesgirls,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-01-24,2020-07-15,2020-07-15,2020-07-15,2020-07-15,2020-07-15,2020-07-15


In [16]:
# impute released
missing_df_released, missing_appids_released = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='released',
    number_consec_panels_missing=1)
print(missing_df_released.shape)

DFT = new_class_obj.impute_missing_using_adj_panels(
    var='released',
    adj_panels=1,
    method='mode')
new_DF = new_class_obj.replace_cols(new_cols=DFT)

number of apps with at least 1 consecutive missing panels for released are 661
out of 10576 apps.
(13, 661)


In [83]:
# impute minInstalls
missing_df_minInstalls, missing_appids_minInstalls = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='minInstalls',
    number_consec_panels_missing=2)
print(missing_df_minInstalls.shape)

DFT = new_class_obj.impute_missing_using_adj_panels(
    var='minInstalls',
    adj_panels=2,
    method='mean')
new_DF = new_class_obj.replace_cols(new_cols=DFT)

number of apps with at least 2 consecutive missing panels for minInstalls are 339
out of 10576 apps.
(13, 339)


In [12]:
# impute score
missing_df_score, missing_appids_score = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='score',
    number_consec_panels_missing=2)

number of apps with at least 2 consecutive missing panels for score are 325
out of 10576 apps.


In [13]:
# impute reviews
missing_df_reviews, missing_appids_reviews = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='reviews',
    number_consec_panels_missing=2)

number of apps with at least 2 consecutive missing panels for reviews are 323
out of 10576 apps.


In [14]:
# impute ratings
missing_df_ratings, missing_appids_ratings = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='ratings',
    number_consec_panels_missing=2)

number of apps with at least 2 consecutive missing panels for ratings are 637
out of 10576 apps.


In [95]:
# impute price
missing_df_price, missing_appids_price = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='price',
    number_consec_panels_missing=2)
print(missing_df_price.shape)

COL_P = new_class_obj.impute_missing_using_adj_panels(
    var='price',
    adj_panels=4,
    method='mean')

number of apps with at least 2 consecutive missing panels for price are 319
out of 10576 apps.
(13, 319)


In [96]:
new_DF = new_class_obj.replace_cols_list(
    list_new_cols=[COL_S, COL_P])

In [29]:
new_class_obj = pre_processing(
                  df=new_DF,
                  df_index = 'appid',
                  initial_panel=initial_panel,
                  all_panels=all_panels,
                  consec_panels=consec_panels)

missing_df, missing_appids = new_class_obj.check_apps_with_consecutive_missing_panels(
    var='updated',
    number_consec_panels_missing=1)
print(missing_df.shape)

number of apps with at least 1 consecutive missing panels for updated are 3
out of 10576 apps.
(13, 3)
